### Import libraries

In [16]:
import pandas as pd
import requests
from bs4 import BeautifulSoup

### CSV

The manager of the loanfirm has some user-specific data that we wants us to enhance with extra information. In the end we should have our user data enhanced with the per capita income on a county level which can be found here:
https://en.wikipedia.org/wiki/List_of_United_States_counties_by_per_capita_income

There are however some steps we need to complete first. First off all the data is scattered over a number of csv files. Let's import all of them and combine them into one masterfile

Use the os module to get all the filesnames

In [18]:
# should add an extra slide in csv's for paths

import os

file_names = os.listdir('../raw_data/adresses')
file_names

['az.csv',
 'wa.csv',
 'nv.csv',
 'or.csv',
 'ak.csv',
 'ca.csv',
 'id.csv',
 'hi.csv',
 'co.csv',
 'nm.csv',
 'mt.csv']

Let's combine them with the proper path so that we can read the csv's.

In [19]:
file_paths = []

for name in file_names:
    file_paths.append('../raw_data/adresses/' + name)
    
file_paths

['../raw_data/adresses/az.csv',
 '../raw_data/adresses/wa.csv',
 '../raw_data/adresses/nv.csv',
 '../raw_data/adresses/or.csv',
 '../raw_data/adresses/ak.csv',
 '../raw_data/adresses/ca.csv',
 '../raw_data/adresses/id.csv',
 '../raw_data/adresses/hi.csv',
 '../raw_data/adresses/co.csv',
 '../raw_data/adresses/nm.csv',
 '../raw_data/adresses/mt.csv']

Now that we have the proper path in a variable we can loop over them to read our csv's. We would however like to save them in one variable all at once. 

Let's loop over all of them and save them in a dictionary structure where 
- **key = filename**
- **value = dataframe**

Use *nrows=10* to limit the size of our dataset

In [ ]:
diction = {}

for name, path in zip(file_names, file_paths):
    diction[name] = pd.read_csv(path, nrows=10)

In [ ]:
for name, path in zip(file_names, file_paths):
    diction[name] = pd.read_csv(path, nrows=10)

We can use **pd.concat()** to concatenate our dataframes into one

In [21]:
full_df = pd.concat(diction.values())

In [22]:
full_df.shape

(110, 11)

In [23]:
full_df.head()

,LON,LAT,NUMBER,STREET,UNIT,CITY,DISTRICT,REGION,POSTCODE,ID,HASH
0,-111.572316,35.236722,6110.0,N HIGHWAY 89,NaN,NaN,NaN,NaN,86004.0,NaN,ec074bdfa4581255
1,-111.672107,35.233690,3111.0,W SHANNON DR,NaN,NaN,NaN,NaN,86001.0,NaN,a7d9e14ded9387d9
2,-111.655159,35.215620,1380.0,N ROCKRIDGE RD,NaN,NaN,NaN,NaN,86001.0,NaN,6483fb4f4c44a89c
3,-111.628072,35.213824,2200.0,N GEMINI DR,NaN,NaN,NaN,NaN,86004.0,NaN,6ebc625bcc95e169
4,-111.602377,35.213094,3101.0,E LEWIS DR,NaN,NaN,NaN,NaN,86004.0,NaN,0de30a0ef7c77856


If we check our null values we see that we're missing quite some data. Also the data is not on the county level. 

In [24]:
full_df.isnull().sum()/len(full_df)

LON         0.000000
LAT         0.000000
NUMBER      0.009091
STREET      0.036364
UNIT        0.900000
CITY        0.263636
DISTRICT    1.000000
REGION      0.727273
POSTCODE    0.454545
ID          0.818182
HASH        0.000000
dtype: float64

### API

The great part is however that the latitude and lonitude are present in all row. So we can use an API to get the county data with these lat and lon columns

Documentation:
https://nominatim.org/release-docs/develop/api/Overview/

Looking at the documentation we spot that we need to use the **endpoint /reverse**

https://nominatim.org/release-docs/develop/api/Reverse/

Let's construct our URL based on this information:

In [25]:
base_url = 'https://nominatim.openstreetmap.org'

endpoint = '/reverse'

full_url = base_url + endpoint

full_url

'https://nominatim.openstreetmap.org/reverse'

We have three params we could use namely **lat** and **lon**, which we will get from our DF, and **format** which will be fixed on json

Let's create a function that will send a *GET-request* for a given LAT and LON value and return the county

In [41]:
import numpy as np

def get_county(lon, lat):
    
    #define params
    params = {
        'lat': lat,
        'lon' : lon,
        'format' : 'json'
    }
    
    #send get request
    response = requests.get(full_url, params).json()
    
    #go through dictionary to get to the county
    if 'county' in response['address'].keys():
        county = response['address']['county']
        
        #replace county for wikipedia mach
        county_clean = county.replace(' County', '')
    else:
        county_clean = np.nan    
    
    #return county_clean
    return county_clean

In [42]:
get_county(-111.5723157, 35.2367222)

'Coconino'

We can now use this function and the **.iterrows()** method to loop over our DF and create an extra columns with the county

In [43]:
county_ls = []

for index, row in full_df.iterrows():
    county = get_county(row['LON'], row['LAT'])
    county_ls.append(county)

Now can add a new column with the county data

In [45]:
full_df['county'] = county_ls
full_df.county.value_counts()

Coconino      10
Whatcom       10
Baker         10
Anchorage     10
Alameda       10
Ada           10
Honolulu      10
Rio Blanco    10
Sandoval       4
Valencia       3
Custer         3
McKinley       2
Deer Lodge     2
Lincoln        2
Eddy           1
Rosebud        1
Toole          1
Sheridan       1
Name: county, dtype: int64

### Scraping

We can use these counties now to get the Per Capita Income

Let's first get a list of all unique counties in our dataset

In [46]:
counties = full_df.county.unique()
counties

array(['Coconino', 'Whatcom', nan, 'Baker', 'Anchorage', 'Alameda', 'Ada',
       'Honolulu', 'Rio Blanco', 'McKinley', 'Sandoval', 'Valencia',
       'Eddy', 'Rosebud', 'Custer', 'Toole', 'Deer Lodge', 'Lincoln',
       'Sheridan'], dtype=object)

Let's inspect our webpage and try to identidy a unique class we could scrape 

In [47]:
url_to_scrape = 'https://en.wikipedia.org/wiki/List_of_United_States_counties_by_per_capita_income'

content = requests.get(url_to_scrape).content

soup = BeautifulSoup(content)

wikitable is the one we need

In [48]:
table = soup.find('table', class_='wikitable')

Inside this table we can use **.find_all()** to get all the **tr** elements. We should get a list of all table elements.

In [49]:
trs = table.find_all('tr')

Check the first **tr** element and try to figure out how we could get the necessary information out of this.

Watch out for the headers and attribute errors

In [50]:
cap_dict = {}

for row in trs[1:]:
    try:
        county = row.a.text
        per_capita = row.find_all('td')[3].text
        per_capita_stripped = int(per_capita.lstrip('$').replace(',',''))
        cap_dict[county] = per_capita_stripped
    except AttributeError:
        pass
        

In [51]:
cap_dict

{'New York County': 76592,
 'Arlington': 62018,
 'Falls Church City': 59088,
 'Marin': 56791,
 'Santa Clara': 56248,
 'Alexandria City': 54608,
 'Pitkin': 51814,
 'Los Alamos': 51044,
 'Fairfax County': 50532,
 'Hunterdon': 50349,
 'Borden': 50042,
 'Montgomery': 15523,
 'Morris': 20045,
 'Fairfield': 20257,
 'San Francisco': 48486,
 'Howard': 18707,
 'Westchester': 47984,
 'Somerset': 16748,
 'Nantucket': 47331,
 'Loudoun': 46565,
 'North Slope': 46457,
 'San Mateo': 45732,
 'Kalawao': 45515,
 'Washington City': 45290,
 'Goochland': 45039,
 'Norfolk': 44692,
 'Fairfax City': 44345,
 'Douglas': 16404,
 'Teton': 22436,
 'Bergen': 43347,
 'Middlesex': 30531,
 'Monmouth': 42749,
 'Summit': 27818,
 'Nassau': 28926,
 'Mineral': 19788,
 'Chester': 18098,
 'Ozaukee': 42041,
 'Clear Creek': 41716,
 'Williamson': 23605,
 'San Miguel': 17926,
 'Anne Arundel': 40415,
 'Putnam': 18377,
 'Newport': 40293,
 'Delaware County': 20854,
 'King': 29836,
 'Fauquier': 39600,
 'Hamilton': 16295,
 'Eagle': 3

We can now use the .map() function to enrich our dataset

In [52]:
full_df['per_capita'] = full_df.county.map(cap_dict)

In [53]:
full_df.per_capita.isnull().sum()

10